In [1]:
# Credit Card Fraud Detection   https://www.kaggle.com/mlg-ulb/creditcardfraud

# Goal of this model to detect the fradulent CC transacations

# LogisticRegression & Random Forest 

In [2]:
# Read the csv file and cache the data
from pyspark.sql import SQLContext

from pyspark import SparkContext

sqlContext = SQLContext(sc)

df = sqlContext.read.csv('/FileStore/tables/creditcard.csv',header='true', inferSchema='true')
df.cache()

Out[ 1 ]: DataFrame[Time: decimal(10,0), V1: double, V2: double, V3: double, V4: double, V5: double, V6: double, V7: double, V8: double, V9: double, V10: double, V11: double, V12: double, V13: double, V14: double, V15: double, V16: double, V17: double, V18: double, V19: double, V20: double, V21: double, V22: double, V23: double, V24: double, V25: double, V26: double, V27: double, V28: double, Amount: double, Class: int]

In [3]:
# Look at the count of distinct records: Normal vs Fraudelent transactions;
df.select("Class").distinct().show()
df.groupBy("Class").count().show()


+-----+
Class|
+-----+
 1|
 0|
+-----+

+-----+------+
Class| count|
+-----+------+
 1| 492|
 0|284315|
+-----+------+

In [4]:
# Drop the Time feature as we are not relying on it
# Rename the class to label for BinaryClassificationEvaluator

df = df.drop("Time")
df = df.withColumnRenamed("Class", "label")

In [5]:
from  pyspark.ml.feature import StandardScaler
from  pyspark.ml.feature import VectorAssembler

# Vectorize the amount column
amt_assembler = VectorAssembler(inputCols = ['Amount'],outputCol = "vec_Amount")

df = amt_assembler.transform(df)




In [6]:
# Now Standardize the Amount column, iternal Stats function 
standardizer = StandardScaler(withMean=True, withStd=True,inputCol='vec_Amount',outputCol='std_Amount')
model = standardizer.fit(df)
df = model.transform(df)

In [7]:
# vectorize all the features columns
sel_cols = ['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','std_Amount']
assembler = VectorAssembler(inputCols = sel_cols,outputCol = "features")
df = assembler.transform(df)

In [8]:
# Split the dataset into training/testing for Random Classifier, seed is justa random sample data fucntion 
(trainingData, testData) = df.randomSplit([0.7, 0.3], seed = 100)
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="label", featuresCol="features")
rfModel = rf.fit(trainingData)
predictions = rfModel.transform(testData)
selected = predictions.select("label", "prediction", "probability", "Amount")
selected.show()

+-----+----------+--------------------+-------+
label|prediction| probability| Amount|
+-----+----------+--------------------+-------+
 0| 0.0|[0.99775801475801...| 2520.0|
 1| 1.0|[0.29922294067656...| 99.99|
 0| 0.0|[0.99572544235796...|4313.82|
 0| 1.0|[0.29922294067656...| 1.0|
 1| 1.0|[0.29922294067656...| 99.99|
 1| 1.0|[0.29922294067656...| 99.99|
 0| 0.0|[0.94298975661087...| 845.73|
 0| 0.0|[0.98809389399204...| 1.0|
 0| 0.0|[0.98797012663074...| 1.0|
 1| 1.0|[0.29922294067656...| 99.99|
 1| 1.0|[0.29922294067656...| 99.99|
 0| 0.0|[0.98861145819002...| 1.0|
 0| 0.0|[0.99337376589921...| 15.0|
 1| 1.0|[0.07170173939863...| 1.0|
 0| 0.0|[0.99738699817502...| 325.56|
 1| 1.0|[0.05975076902069...| 1.0|
 1| 1.0|[0.05975076902069...| 1.0|
 0| 0.0|[0.99914839841505...| 99.99|
 0| 0.0|[0.99914839841505...| 99.99|
 1| 1.0|[0.07170173939863...| 1.0|
+-----+----------+--------------------+-------+
only showing top 20 rows

In [9]:
# Now evaluate the model 
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})

Out[ 8 ]: 0.8086795456779414

In [10]:
evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

Out[ 9 ]: 0.979717113387102

In [11]:
# Now build the Logistic Regression 
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})


Out[ 10 ]: 0.9768333963546695

In [12]:
evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})
# Area Under PR is less for logistic Regression; hence RandomForest is the best model

Out[ 11 ]: 0.763135898517088

In [13]:
# Cross validate the RandomForest model to get best model
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 4, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .build())
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid,
evaluator=evaluator, numFolds=10)
cvModel = cv.fit(trainingData)
predictions = cvModel.transform(testData)
evaluator.evaluate(predictions)



In [14]:
# Area under PR is slighly improved, let's pick the bestModel
evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})


Out[ 15 ]: 0.8080410567362993

In [15]:
bestModel = cvModel.bestModel
finalPredictions = bestModel.transform(df)
evaluator.evaluate(finalPredictions)

Out[ 16 ]: 0.9800805859297558

In [16]:
evaluator.evaluate(finalPredictions, {evaluator.metricName: "areaUnderPR"})



Out[ 17 ]: 0.8075707132031913